In [1]:
import torch
import treescope
from models.gan import DinoPatchDiscriminator
from torchmetrics.image import (
    StructuralSimilarityIndexMeasure,
    PeakSignalNoiseRatio,
    FrechetInceptionDistance,
    LearnedPerceptualImagePatchSimilarity,
)

treescope.register_as_default()

/home/jajal/mambaforge/envs/minVAE/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model = DinoPatchDiscriminator('small')
model(torch.randn(1, 3, 256, 256).cuda())

Using cache found in /home/jajal/.cache/torch/hub/facebookresearch_dinov2_main


DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-11): 12 x NestedTensorBlock(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )
  )
  (norm): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
  (head): Identity()
)

In [2]:
from layers.layers2d import Encoder, Decoder
from models.vae import VAE

In [ ]:
vae = VAE(
    in_channels=3,
    out_channels=3,
    channels=128,
    channels_mult=[1, 2, 4, 4],
    num_res_blocks=2,
    attn_resolutions=[32],
    dropout=0.0,
    resolution=256,
    z_channels=4,
    spatial_compression=4,
    wavelet="db4",
    maxlevel=2
)
vae

z of shape: (1, 4, 16, 16), dimensions: 1024


VAE(
  (wavelet_transform): WaveletTransform()
  (encoder): Encoder(
    (conv_in): Conv2d(48, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down): ModuleList(
      (0): Module(
        (block): ModuleList(
          (0-1): 2 x ResnetBlock(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nin_shortcut): Identity()
          )
        )
        (attn): ModuleList()
        (downsample): Downsample(
          (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
        )
      )
      (1): Module(
        (block): ModuleList(
          (0): ResnetBlock(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 256, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nin_shortcut): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))
          )
          (1): ResnetBlock(
            (norm1): GroupNorm(32, 256, eps=1e-06, affine=True)
            (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 256, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nin_shortcut): Identity()
          )
        )
        (attn): ModuleList(
          (0-1): 2 x AttnBlock(
            (norm): GroupNorm(32, 256, eps=1e-06, affine=True)
            (qkv): Conv2d(256, 768, kernel_size=(1, 1), stride=(1, 1))
            (proj_out): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
          )
        )
        (downsample): Downsample(
          (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2))
        )
      )
      (2): Module(
        (block): ModuleList(
          (0): ResnetBlock(
            (norm1): GroupNorm(32, 256, eps=1e-06, affine=True)
            (conv1): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 512, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nin_shortcut): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1))
          )
          (1): ResnetBlock(
            (norm1): GroupNorm(32, 512, eps=1e-06, affine=True)
            (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 512, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nin_shortcut): Identity()
          )
        )
        (attn): ModuleList()
      )
      (3): Module(
        (block): ModuleList(
          (0-1): 2 x ResnetBlock(
            (norm1): GroupNorm(32, 512, eps=1e-06, affine=True)
            (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 512, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nin_shortcut): Identity()
          )
        )
        (attn): ModuleList()
      )
    )
    (mid): Module(
      (block1): ResnetBlock(
        (norm1): GroupNorm(32, 512, eps=1e-06, affine=True)
        (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (norm2): GroupNorm(32, 51

In [16]:
vae.forward(torch.randn(1, 3, 256, 256))

torch.Size([1, 4, 16, 16])


(tensor([[[[-0.1308, -0.1195, -0.0970,  ...,  0.0094,  0.1087, -0.0962],
           [-0.1310, -0.1170, -0.0949,  ..., -0.0015,  0.1147, -0.0974],
           [-0.1075, -0.0862, -0.0610,  ...,  0.0070,  0.1276, -0.1109],
           ...,
           [-0.0429, -0.0621, -0.1307,  ..., -0.0038,  0.0180, -0.0152],
           [ 0.0218,  0.0133, -0.0084,  ...,  0.0095,  0.0100, -0.0106],
           [-0.0050,  0.0088,  0.0508,  ..., -0.0075, -0.0135,  0.0136]],
 
          [[-0.0317, -0.0255, -0.0152,  ...,  0.0168,  0.0391, -0.0392],
           [-0.0442, -0.0294, -0.0081,  ...,  0.0015,  0.0298, -0.0260],
           [-0.0767, -0.0438, -0.0017,  ..., -0.0207, -0.0023,  0.0087],
           ...,
           [ 0.0179,  0.0165,  0.0106,  ...,  0.2048, -0.0646, -0.0157],
           [ 0.0282,  0.0446,  0.0663,  ...,  0.0661, -0.0006, -0.0238],
           [-0.0310, -0.0455, -0.0634,  ..., -0.1211,  0.0240,  0.0228]],
 
          [[-0.0438, -0.0415, -0.0334,  ...,  0.1213, -0.0159, -0.0283],
           [-0.0318, -0.0319, -0.0339,  ...,  0.1611,  0.0049, -0.0589],
           [ 0.0050,  0.0011, -0.0179,  ...,  0.2797,  0.0606, -0.1451],
           ...,
           [ 0.0201,  0.0342,  0.0463,  ..., -0.1936, -0.0917,  0.1440],
           [-0.0496, -0.0583, -0.0718,  ..., -0.0674, -0.0377,  0.0537],
           [ 0.0363,  0.0386,  0.0458,  ...,  0.1272,  0.0618, -0.0947]]]],
        grad_fn=<AddBackward0>),
 (tensor([[[[-1.2924e-01, -1.6502e-01, -3.0950e-01,  ..., -2.7536e-01,
             -1.2552e-01, -1.1578e-01],
            [ 5.4256e-02, -5.1158e-02,  5.0583e-02,  ...,  8.9288e-02,
             -8.6735e-02,  2.0966e-01],
            [-1.2223e-01,  4.2466e-01,  4.7688e-01,  ..., -5.6621e-01,
             -5.1838e-03,  2.6259e-01],
            ...,
            [ 6.6059e-02, -4.4909e-02,  6.6881e-01,  ...,  8.2845e-02,
              1.6433e-01, -3.9653e-02],
            [ 2.5342e-01,  4.0554e-02,  8.0081e-01,  ...,  1.5986e-02,
              7.4698e-02,  8.0354e-02],
            [ 2.7520e-01,  1.1578e-01,  2.9482e-01,  ...,  4.1454e-03,
             -2.8699e-02,  5.1814e-02]],
  
           [[-2.0334e-01,  6.9834e-02, -2.8804e-03,  ..., -3.1529e-01,
              1.3594e-01, -4.5930e-02],
            [ 3.2392e-01,  7.8728e-02,  1.6393e-01,  ...,  1.4608e-02,
             -1.9419e-01, -3.4619e-01],
            [-7.3269e-02, -2.5476e-01, -1.7885e-01,  ...,  1.1333e-01,
             -1.8090e-01, -2.8775e-02],
            ...,
            [ 1.3140e-01,  2.5361e-01,  3.4192e-01,  ...,  3.8019e-01,
              3.1558e-01,  1.9549e-02],
            [-3.0887e-02,  6.2719e-02, -4.2411e-01,  ...,  5.7170e-01,
             -7.2368e-02,  3.7075e-02],
            [ 1.6372e-01,  2.0456e-01,  2.8146e-01,  ..., -2.5582e-01,
              1.3022e-01,  1.7702e-01]],
  
           [[ 1.0269e-01, -1.2490e-02, -5.2652e-02,  ...,  1.3487e-01,
              1.0079e-01, -2.5841e-02],
            [-1.7434e-01,  1.5141e-01,  1.8581e-01,  ...,  1.4136e-01,
             -1.2862e-02,  4.8371e-01],
            [-4.7961e-02,  2.5470e-02,  7.8326e-03,  ..., -1.9214e-01,
              2.3683e-01,  1.1144e-01],
            ...,
            [ 3.7207e-01,  4.3878e-02,  3.3695e-01,  ...,  5.1172e-01,
             -3.8362e-02,  2.4738e-01],
            [ 4.0233e-02, -1.0129e-03,  2.0766e-01,  ...,  4.3456e-01,
              1.4562e-01,  2.3929e-01],
            [-1.9190e-01, -7.1105e-04, -3.9535e-02,  ...,  4.7775e-01,
              2.3119e-01,  1.9029e-01]],
  
           [[-4.0796e-01, -2.3980e-01, -2.1413e-01,  ...,  1.3493e-01,
              1.2236e-01, -2.2008e-02],
            [-3.4870e-01,  1.1583e-01, -3.1384e-01,  ...,  6.3484e-04,
              1.0258e-02,  3.1069e-02],
            [-1.2024e-01, -3.3120e-01, -1.9441e-01,  ...,  1.2150e-01,
              5.5513e-02,  2.0165e-01],
            ...,
            [-1.8271e-01, -2.5059e-01,  5.1623e-01,  ...,  2.1971e-01,
             -6.5691e-03, -1.0982e-01],
            [-1.7819e-02, -1.3981e-01, -8.9913e-02,  ..., -3.8752e-01,
   

In [ ]:
encoder = Encoder(
    in_channels=3,
    channels=128,
    channels_mult=[1, 2, 4, 4],
    num_res_blocks=2,
    attn_resolutions=[32],
    resolution=256,
    z_channels=4,
    spatial_compression=8,
    dropout=0.1,
)

decoder = Decoder(
    z_channels=4,
    channels=128,
    channels_mult=[1, 2, 4, 4],
    num_res_blocks=2,
    attn_resolutions=[32],
    resolution=256,
    out_channels=3,
    spatial_compression=8,
    dropout=0.1,
)

In [ ]:
encoder

In [ ]:
decoder

In [ ]:
decoder.forward(torch.randn(1, 4, 8, 8)).shape